In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
socar_zone = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/socar_zone_recommendation.csv')

In [ ]:
# age_Categorizing
conditionlist = [
                 (socar_zone['member_age']<30),
                 (socar_zone['member_age']>=30) & (socar_zone['member_age']<40),
                 (socar_zone['member_age']>=40) & (socar_zone['member_age']<50),
                 (socar_zone['member_age']>=50) & (socar_zone['member_age']<60),
                 (socar_zone['member_age']>=60) & (socar_zone['member_age']<70),
                 (socar_zone['member_age']>=70)
                 ]
choicelist = ['20', '30', '40', '50', '60', '70']
socar_zone['age_category']=np.select(conditionlist, choicelist, default='Not specified')


In [ ]:
# car_Categorizing
conditionlist = [
             (socar_zone['car_name']=='아반떼AD') | (socar_zone['car_name']=='올뉴K3') | (socar_zone['car_name']=='더뉴아반떼') | (socar_zone['car_name']=='Thule 자전거캐리어 X 더뉴아반떼') | (socar_zone['car_name']=='[쏘카세이브] 아반떼AD'),
             (socar_zone['car_name']=='스포티지') | (socar_zone['car_name']=='투싼(경유)') | (socar_zone['car_name']=='쏘렌토') | (socar_zone['car_name']=='스포티지 더볼드') | (socar_zone['car_name']=='싼타페') | (socar_zone['car_name']=='아이오닉EV (제주)') | (socar_zone['car_name']=='재규어 E-Pace') | (socar_zone['car_name']=='[쏘카세이브] 투싼(경유)'),
             (socar_zone['car_name']=='카니발 11인승') | (socar_zone['car_name']=='휠체어슬로프 X 카니발'),
             (socar_zone['car_name']=='그랜저IG (LPG)') | (socar_zone['car_name']=='그랜저HG') | (socar_zone['car_name']=='그랜저HG (경유)') | (socar_zone['car_name']=='G8'),
             (socar_zone['car_name']=='코나') | (socar_zone['car_name']=='티볼리(경유)') | (socar_zone['car_name']=='티볼리') | (socar_zone['car_name']=='지프 레니게이드') | (socar_zone['car_name']=='스토닉(경유)') | (socar_zone['car_name']=='스토닉') | (socar_zone['car_name']=='Thule 루프박스 X 티볼리') | (socar_zone['car_name']=='QM3') | (socar_zone['car_name']=='트랙스') | (socar_zone['car_name']=='[쏘카세이브] 티볼리(경유)'),
             (socar_zone['car_name']=='클리오') | (socar_zone['car_name']=='프라이드') | (socar_zone['car_name']=='볼트EV (제주)') | (socar_zone['car_name']=='미니클럽맨') | (socar_zone['car_name']=='볼트EV') | (socar_zone['car_name']=='[쏘카세이브] 프라이드'),
             (socar_zone['car_name']=='더뉴레이') | (socar_zone['car_name']=='넥스트스파크') | (socar_zone['car_name']=='올뉴모닝') | (socar_zone['car_name']=='레이') | (socar_zone['car_name']=='모닝') | (socar_zone['car_name']=='펫카시트 X 더뉴레이') | (socar_zone['car_name']=='특가 더뉴레이') | (socar_zone['car_name']=='[쏘카세이브] 레이') | (socar_zone['car_name']=='[쏘카세이브] 넥스트스파크'),
             (socar_zone['car_name']=='말리부') | (socar_zone['car_name']=='SM6') | (socar_zone['car_name']=='쏘나타 뉴라이즈(LPG)') | (socar_zone['car_name']=='K5 (LPG)') | (socar_zone['car_name']=='스팅어') | (socar_zone['car_name']=='쏘나타 뉴라이즈') | (socar_zone['car_name']=='K5') | (socar_zone['car_name']=='2016 말리부') | (socar_zone['car_name']=='LF쏘나타') | (socar_zone['car_name']=='벤츠 C200') | (socar_zone['car_name']=='쏘나타DN8') | (socar_zone['car_name']=='[쏘카세이브] K5') | (socar_zone['car_name']=='[쏘카세이브] K5 (LPG)'),
             (socar_zone['car_name']=='뉴 그랜드스타렉스') | (socar_zone['car_name']=='스타렉스')]

choicelist = ['준중형', '중형SUV', '대형MPV', '준대형', '소형SUV', '소형', '경차', '중형', '승합차']


socar_zone['car_category']=np.select(conditionlist, choicelist, default='Not Specified')


In [ ]:
socar_zone.head()

,reservation_id,region,reservation_return_at,reservation_start_at,member_age,member_gender,car_name,zone_name,zone_address,zone_lat,zone_lng,zone_type1,zone_type2,zone_type3,reservation_created_lat,reservation_created_lng,age_category,car_category
0,19600422,서울특별시 강서구,2019-01-01 17:33:42,2018-12-31 17:30:00,48,male,아반떼AD,SK주차장,서울 강서구 방화동 500-8,37.573011,126.804605,LIVING_APT,TRANSFER_STATION,TRANSFER_SUBWAY,NaN,NaN,40,준중형
1,19637831,울산광역시 남구,2019-01-01 12:08:55,2018-12-31 16:00:00,25,male,스포티지,우정유료주차장,울산 남구 삼산동 1481-10,35.536034,129.339783,TRANSFER_TERMINAL,TRANSFER_TERMINAL,COMMERCIAL_HOTSPOT,NaN,NaN,20,중형SUV
2,19670646,경기도 김포시,2019-01-02 17:30:00,2019-01-02 14:50:00,34,male,카니발 11인승,월드애비뉴,경기 김포시 구래동 6880-7,37.645571,126.627879,LIVING_APT,COMMERCIAL_HOTSPOT,ETC,NaN,NaN,30,대형MPV
3,19692831,경기도 남양주시,2019-01-01 20:53:09,2018-12-31 11:00:00,23,male,아반떼AD,제뉴빌딩주차장,경기 남양주시 호평동 642-2,37.654861,127.244576,LIVING_APT,TRANSFER_SUBWAY,COMMERCIAL_HOTSPOT,NaN,NaN,20,준중형
4,19728911,서울특별시 강서구,2019-01-04 14:06:03,2019-01-03 08:00:00,35,male,투싼(경유),용정주차장,서울 강서구 방화동 608-9,37.565186,126.816185,LIVING_ETC,TRANSFER_SUBWAY,ETC,NaN,NaN,30,중형SUV


In [ ]:
# Ordianl Encoding
enc = OrdinalEncoder(dtype=np.float64)
ordinal1 = enc.fit_transform(np.asarray(socar_zone['zone_type1']).reshape(-1,1))
socar_zone['zone_type1'] = ordinal1[:,0]
ordinal2 = enc.fit_transform(np.asarray(socar_zone['zone_type2']).reshape(-1,1))
socar_zone['zone_type2'] = ordinal2[:,0]
ordinal3 = enc.fit_transform(np.asarray(socar_zone['zone_type3']).reshape(-1,1))
socar_zone['zone_type3'] = ordinal3[:,0]
ordinal4 = enc.fit_transform(np.asarray(socar_zone['member_gender']).reshape(-1,1))
socar_zone['member_gender'] = ordinal4[:,0]
ordinal5 = enc.fit_transform(np.asarray(socar_zone['region']).reshape(-1,1))
socar_zone['region_ordinal'] = ordinal5[:,0]
ordinal6 = enc.fit_transform(np.asarray(socar_zone['car_name']).reshape(-1,1))
socar_zone['car_name'] = ordinal6[:,0]
ordinal7 = enc.fit_transform(np.asarray(socar_zone['zone_name']).reshape(-1,1))
socar_zone['zone_name'] = ordinal7[:,0]
ordinal8 = enc.fit_transform(np.asarray(socar_zone['car_category']).reshape(-1,1))
socar_zone['car_category'] = ordinal8[:,0]

In [ ]:
socar_zone['distance'] = ((socar_zone['zone_lat']-socar_zone['reservation_created_lat'])**2 + (socar_zone['zone_lng']-socar_zone['reservation_created_lng'])**2)*100000

In [ ]:

##이용시간 계산
dataframe = pd.DataFrame()

dataframe['return'] = pd.to_datetime(socar_zone['reservation_return_at'])
dataframe['start'] = pd.to_datetime(socar_zone['reservation_start_at'])

socar_zone['duration'] = dataframe['return'] - dataframe['start']

socar_zone['duration'] = socar_zone['duration'].astype('int64')/1e9

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: casting timedelta64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
socar_zone = socar_zone.drop(['reservation_id','region','reservation_return_at','reservation_start_at','zone_lat','zone_lng','reservation_created_lat','reservation_created_lng'],1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
socar_zone.head()

,member_age,member_gender,car_name,zone_name,zone_address,zone_type1,zone_type2,zone_type3,age_category,car_category,region_ordinal,distance,duration
0,48,1.0,43.0,71.0,서울 강서구 방화동 500-8,5.0,12.0,12.0,40,7.0,18.0,NaN,86622.0
1,25,1.0,36.0,799.0,울산 남구 삼산동 1481-10,13.0,14.0,0.0,20,9.0,23.0,NaN,72535.0
2,34,1.0,49.0,820.0,경기 김포시 구래동 6880-7,5.0,0.0,5.0,30,2.0,5.0,NaN,9600.0
3,23,1.0,43.0,957.0,경기 남양주시 호평동 642-2,5.0,13.0,0.0,20,7.0,6.0,NaN,121989.0
4,35,1.0,52.0,782.0,서울 강서구 방화동 608-9,6.0,13.0,5.0,30,9.0,18.0,NaN,108363.0


In [ ]:
socar_zone = socar_zone.drop(['zone_address'],1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
socar_zone.fillna(0)

,member_age,member_gender,car_name,zone_name,zone_type1,zone_type2,zone_type3,age_category,car_category,region_ordinal,distance,duration
0,48,1.0,43.0,71.0,5.0,12.0,12.0,40,7.0,18.0,0.000000,86622.0
1,25,1.0,36.0,799.0,13.0,14.0,0.0,20,9.0,23.0,0.000000,72535.0
2,34,1.0,49.0,820.0,5.0,0.0,5.0,30,2.0,5.0,0.000000,9600.0
3,23,1.0,43.0,957.0,5.0,13.0,0.0,20,7.0,6.0,0.000000,121989.0
4,35,1.0,52.0,782.0,6.0,13.0,5.0,30,9.0,18.0,0.000000,108363.0
...,...,...,...,...,...,...,...,...,...,...,...,...
751544,26,1.0,13.0,253.0,6.0,3.0,0.0,20,1.0,17.0,0.042377,3674.0
751545,28,0.0,23.0,859.0,5.0,8.0,0.0,20,1.0,7.0,0.302336,2474.0
751546,23,1.0,24.0,704.0,6.0,13.0,0.0,20,7.0,13.0,0.007081,2238.0
751547,30,1.0,14.0,680.0,6.0,11.0,5.0,30,7.0,20.0,0.069138,1868.0


In [ ]:
socar_zone.shape
socar_zone.dtypes

member_age          int64
member_gender     float64
car_name          float64
zone_name         float64
zone_type1        float64
zone_type2        float64
zone_type3        float64
age_category       object
car_category     float64
region_ordinal    float64
distance          float64
duration          float64
dtype: object

In [ ]:
## train & test data
socar_zone = socar_zone.sample(frac=1, random_state=0).reset_index(drop=True)
train = socar_zone[:601240]
test = socar_zone[601240:]

In [ ]:
x_train = np.asarray(train.drop('duration',1)) ## input
y_train = np.asarray(train['duration']) ## target
x_test = np.asarray(test.drop('duration',1)) ## input
y_test = np.asarray(test['duration']) ## target
duration_features = [x for i,x in enumerate(socar_zone.columns) if i!=0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
duration_features

['member_gender',
 'car_name',
 'zone_name',
 'zone_type1',
 'zone_type2',
 'zone_type3',
 'age_category',
 '\x08car_category',
 'region_ordinal',
 'distance',
 'duration']

In [ ]:
# Hyperparameter 조합 정의
param_grid = {'max_iter': [100, 200, 300],
              'max_depth': [2, 3, 4, 5],
              'min_samples_leaf': [1, 20, 100],
              'learning_rate': [0.1, 0.01, 0.05]
              }

In [ ]:
# grid search 모델 정의, 학습 및 model selection
grid_search = GridSearchCV(estimator=ensemble.HistGradientBoostingRegressor(),
                           param_grid=param_grid,
                           cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(x_train, y_train)
print(grid_search.best_params_)
reg = ensemble.HistGradientBoostingRegressor(**grid_search.best_params_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
{'learning_rate': 0.1, 'max_depth': 5, 'max_iter': 300, 'min_samples_leaf': 100}


In [ ]:
from sklearn.model_selection import PredefinedSplit

# train set과 test set의 input과 output를 각각 이어 붙여서 X,y를 정의
X = np.concatenate((x_train,x_test), axis=0)
y = np.concatenate((y_train,y_test), axis=0)


In [ ]:
pds = PredefinedSplit(test_fold=[-1]*len(x_train)+[0]*len(x_test))
# grid search 모델 정의, 학습 및 model selection
grid_search = GridSearchCV(estimator=ensemble.HistGradientBoostingRegressor(),
                           param_grid=param_grid,
                           cv=pds, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X, y)
print(grid_search.best_params_)

Fitting 1 folds for each of 108 candidates, totalling 108 fits
{'learning_rate': 0.1, 'max_depth': 2, 'max_iter': 300, 'min_samples_leaf': 1}


In [ ]:
reg.fit(x_train, y_train)

mse = mean_squared_error(y_test, reg.predict(x_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

mae = mean_absolute_error(y_test, reg.predict(x_test))
print("The mean absolute error (MAE) on test set: {:.4f}".format(mae))

The mean squared error (MSE) on test set: 1977194417.2441
The mean absolute error (MAE) on test set: 23626.8440


In [ ]:
x_train2 = np.asarray(train[['age_category', 'member_gender']]) ## input
y_train2 = np.asarray(train['car_category']) ## target
x_test2 = np.asarray(test[['age_category', 'member_gender']]) ## input
y_test2 = np.asarray(test['car_category']) ## target
# duration_features = [x for i,x in enumerate(socar_zone.columns) if i!=0]
# grid search 모델 정의, 학습 및 model selection
grid_search = GridSearchCV(estimator=ensemble.HistGradientBoostingRegressor(),
                           param_grid=param_grid,
                           cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(x_train2, y_train2)
print(grid_search.best_params_)
reg = ensemble.HistGradientBoostingRegressor(**grid_search.best_params_)
# train set과 test set의 input과 output를 각각 이어 붙여서 X,y를 정의
X = np.concatenate((x_train2,x_test2), axis=0)
y = np.concatenate((y_train2,y_test2), axis=0)
pds = PredefinedSplit(test_fold=[-1]*len(x_train2)+[0]*len(x_test2))
# grid search 모델 정의, 학습 및 model selection
grid_search = GridSearchCV(estimator=ensemble.HistGradientBoostingRegressor(),
                           param_grid=param_grid,
                           cv=pds, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X, y)
print(grid_search.best_params_)
reg.fit(x_train2, y_train2)

mse = mean_squared_error(y_test2, reg.predict(x_test2))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

mae = mean_absolute_error(y_test2, reg.predict(x_test2))
print("The mean absolute error (MAE) on test set: {:.4f}".format(mae))

Fitting 3 folds for each of 108 candidates, totalling 324 fits
{'learning_rate': 0.05, 'max_depth': 5, 'max_iter': 200, 'min_samples_leaf': 1}
Fitting 1 folds for each of 108 candidates, totalling 108 fits
{'learning_rate': 0.05, 'max_depth': 4, 'max_iter': 200, 'min_samples_leaf': 20}
The mean squared error (MSE) on test set: 8.3025
The mean absolute error (MAE) on test set: 2.5716


In [ ]:
x_train3 = np.asarray(train[['distance', 'duration']]) ## input
y_train3 = np.asarray(train['car_category']) ## target
x_test3 = np.asarray(test[['distance', 'duration']]) ## input
y_test3 = np.asarray(test['car_category']) ## target
# duration_features = [x for i,x in enumerate(socar_zone.columns) if i!=0]
# grid search 모델 정의, 학습 및 model selection
grid_search = GridSearchCV(estimator=ensemble.HistGradientBoostingRegressor(),
                           param_grid=param_grid,
                           cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(x_train3, y_train3)
print(grid_search.best_params_)
reg = ensemble.HistGradientBoostingRegressor(**grid_search.best_params_)
# train set과 test set의 input과 output를 각각 이어 붙여서 X,y를 정의
X = np.concatenate((x_train3,x_test3), axis=0)
y = np.concatenate((y_train3,y_test3), axis=0)
pds = PredefinedSplit(test_fold=[-1]*len(x_train3)+[0]*len(x_test3))
# grid search 모델 정의, 학습 및 model selection
grid_search = GridSearchCV(estimator=ensemble.HistGradientBoostingRegressor(),
                           param_grid=param_grid,
                           cv=pds, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X, y)
print(grid_search.best_params_)
reg.fit(x_train3, y_train3)

mse = mean_squared_error(y_test3, reg.predict(x_test3))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

mae = mean_absolute_error(y_test3, reg.predict(x_test3))
print("The mean absolute error (MAE) on test set: {:.4f}".format(mae))

Fitting 3 folds for each of 108 candidates, totalling 324 fits
{'learning_rate': 0.1, 'max_depth': 3, 'max_iter': 300, 'min_samples_leaf': 100}
Fitting 1 folds for each of 108 candidates, totalling 108 fits
{'learning_rate': 0.1, 'max_depth': 5, 'max_iter': 300, 'min_samples_leaf': 1}
The mean squared error (MSE) on test set: 8.3389
The mean absolute error (MAE) on test set: 2.5844


In [ ]:
x_train4 = np.asarray(train[['member_gender','car_category', 'duration','distance']]) ## input
y_train4 = np.asarray(train['age_category']) ## target
x_test4 = np.asarray(test[['member_gender','car_category', 'duration','distance']]) ## input
y_test4 = np.asarray(test['age_category']) ## target
# duration_features = [x for i,x in enumerate(socar_zone.columns) if i!=0]
# grid search 모델 정의, 학습 및 model selection
grid_search = GridSearchCV(estimator=ensemble.HistGradientBoostingRegressor(),
                           param_grid=param_grid,
                           cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(x_train4, y_train4)
print(grid_search.best_params_)
reg = ensemble.HistGradientBoostingRegressor(**grid_search.best_params_)
# train set과 test set의 input과 output를 각각 이어 붙여서 X,y를 정의
X = np.concatenate((x_train4,x_test4), axis=0)
y = np.concatenate((y_train4,y_test4), axis=0)
pds = PredefinedSplit(test_fold=[-1]*len(x_train4)+[0]*len(x_test4))
# grid search 모델 정의, 학습 및 model selection
grid_search = GridSearchCV(estimator=ensemble.HistGradientBoostingRegressor(),
                           param_grid=param_grid,
                           cv=pds, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X, y)
print(grid_search.best_params_)
reg.fit(x_train4, y_train4)

mse = mean_squared_error(y_test4, reg.predict(x_test4))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

mae = mean_absolute_error(y_test4, reg.predict(x_test4))
print("The mean absolute error (MAE) on test set: {:.4f}".format(mae))

Fitting 3 folds for each of 108 candidates, totalling 324 fits
{'learning_rate': 0.1, 'max_depth': 5, 'max_iter': 300, 'min_samples_leaf': 20}
Fitting 1 folds for each of 108 candidates, totalling 108 fits
{'learning_rate': 0.1, 'max_depth': 5, 'max_iter': 300, 'min_samples_leaf': 20}
The mean squared error (MSE) on test set: 70.4240
The mean absolute error (MAE) on test set: 6.8549


In [ ]:
x_train5 = np.asarray(train[['member_gender','age_category', 'duration','distance']]) ## input
y_train5 = np.asarray(train['car_category']) ## target
x_test5 = np.asarray(test[['member_gender','age_category', 'duration','distance']]) ## input
y_test5 = np.asarray(test['car_category']) ## target
# duration_features = [x for i,x in enumerate(socar_zone.columns) if i!=0]
# grid search 모델 정의, 학습 및 model selection
grid_search = GridSearchCV(estimator=ensemble.HistGradientBoostingRegressor(),
                           param_grid=param_grid,
                           cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(x_train5, y_train5)
print(grid_search.best_params_)
reg = ensemble.HistGradientBoostingRegressor(**grid_search.best_params_)
# train set과 test set의 input과 output를 각각 이어 붙여서 X,y를 정의
X = np.concatenate((x_train5,x_test5), axis=0)
y = np.concatenate((y_train5,y_test5), axis=0)
pds = PredefinedSplit(test_fold=[-1]*len(x_train5)+[0]*len(x_test5))
# grid search 모델 정의, 학습 및 model selection
grid_search = GridSearchCV(estimator=ensemble.HistGradientBoostingRegressor(),
                           param_grid=param_grid,
                           cv=pds, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X, y)
print(grid_search.best_params_)
reg.fit(x_train5, y_train5)

mse = mean_squared_error(y_test5, reg.predict(x_test5))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

mae = mean_absolute_error(y_test5, reg.predict(x_test5))
print("The mean absolute error (MAE) on test set: {:.4f}".format(mae))

Fitting 3 folds for each of 108 candidates, totalling 324 fits
{'learning_rate': 0.05, 'max_depth': 4, 'max_iter': 300, 'min_samples_leaf': 20}
Fitting 1 folds for each of 108 candidates, totalling 108 fits
{'learning_rate': 0.1, 'max_depth': 5, 'max_iter': 200, 'min_samples_leaf': 20}
The mean squared error (MSE) on test set: 8.2544
The mean absolute error (MAE) on test set: 2.5627


1. train4,5 비교시 차종에 따른 성별, 연령대, 빌린기간, (이동)거리에 더 상관관계가 있음을 알 수 있다.
따라서 쿠폰을 발급할 때 유저의 성별, 연령대, 빌린기간, 거리를 기준으로 차종을 할인해주는 쿠폰을 발급해주는 방식이 효율적으로 쿠폰사용을 유도할 수 있을 것 같다.

2. 더 자세히 나눠서 유저의 성별과 연령대 or 렌트 기간과 이동거리에 따른 차종예측을 실행하였고, 렌트기간과 이동거리보다 이용자의 성별과 연령대가 유저가 어떤 차를 사용할지 예측하는데 더 적은 에러가 났다

(2번은 잘 모르겠어요. train5보다 에러가 조금 높아서 더 자세히 나누는게 필요없나 싶습니다.)
